In [1]:
import pandas as pd
import numpy as np

### Reading in data of all australian crimes by local government area, suburb and offences

In [2]:
aus_crime_df = pd.read_excel('../000_Resources/Crime rate/table3_criminal_incidents_offence_lga_postcode_suburb_oct2010 _sept2020.xlsx')

In [3]:
aus_crime_df

,Year,Year ending,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2020,September,Alpine,3691,Dederang,B Property and deception offences,B40 Theft,B41 Motor vehicle theft,1
1,2020,September,Alpine,3691,Glen Creek,C Drug offences,C30 Drug use and possession,C32 Drug possession,1
2,2020,September,Alpine,3691,Glen Creek,F Other offences,F90 Miscellaneous offences,F93 Cruelty to animals,1
3,2020,September,Alpine,3691,Gundowring,B Property and deception offences,B40 Theft,B49 Other theft,1
4,2020,September,Alpine,3691,Gundowring,E Justice procedures offences,E10 Justice procedures,E13 Resist or hinder officer,1
...,...,...,...,...,...,...,...,...,...
315466,2011,September,Yarriambiack,3489,Tempy,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,1
315467,2011,September,Yarriambiack,3489,Tempy,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A732 Non-FV Threatening behaviour,1
315468,2011,September,Yarriambiack,3489,Tempy,B Property and deception offences,B20 Property damage,B21 Criminal damage,3
315469,2011,September,Yarriambiack,3489,Tempy,D Public order and security offences,D10 Weapons and explosives offences,D11 Firearms offences,1


### used a group by to filter for relevant data

In [4]:
aus_crime_df_group = pd.DataFrame(aus_crime_df.groupby(['Year','Local Government Area','Suburb/Town Name','Postcode'])['Incidents Recorded'].sum())

### reseting index

In [5]:
aus_crime_df_group = aus_crime_df_group.reset_index()

### renaming columns

In [6]:
aus_crime_df_group.columns = ['year','lga','suburb','postcode','total_incidents']


### Making all string values lower case

In [29]:
aus_crime_df_group['lga'] = aus_crime_df_group['lga'].str.lower()
aus_crime_df_group['suburb'] = aus_crime_df_group['suburb'].str.lower()
aus_crime_df_group

,year,lga,suburb,postcode,total_incidents
0,2011,alpine,abbeyard,3737,2
1,2011,alpine,barwidgee,3737,1
2,2011,alpine,bright,3741,68
3,2011,alpine,buckland,3740,2
4,2011,alpine,buffalo river,3737,2
...,...,...,...,...,...
24201,2020,yarriambiack,speed,3488,2
24202,2020,yarriambiack,wallup,3401,1
24203,2020,yarriambiack,warracknabeal,3393,178
24204,2020,yarriambiack,woomelang,3485,2


### reading in suburbs of greater melbourne csv 

In [26]:
melb_suburbs_df = pd.read_csv('01_Outputs/melbourne_suburbs.csv')
melb_suburbs_df['suburb'] = melb_suburbs_df['suburb'].str.lower()
melb_suburbs_df

,suburb
0,abbotsford
1,aberfeldie
2,airport west
3,albanvale
4,albert park
...,...
317,williamstown
318,williamstown north
319,windsor
320,yallambie


### merging aus_crime_df_group with melb_suburbs_df for only melbourne suburbs

In [30]:
merged_melb_subs = aus_crime_df_group.merge(melb_suburbs_df, on = 'suburb', how = 'inner')
merged_melb_subs

,year,lga,suburb,postcode,total_incidents
0,2011,banyule,bellfield,3081,93
1,2012,banyule,bellfield,3081,110
2,2013,banyule,bellfield,3081,104
3,2014,banyule,bellfield,3081,105
4,2015,banyule,bellfield,3081,81
...,...,...,...,...,...
3757,2016,yarra ranges,wandin north,3139,86
3758,2017,yarra ranges,wandin north,3139,75
3759,2018,yarra ranges,wandin north,3139,80
3760,2019,yarra ranges,wandin north,3139,59


### filtering for specific year

In [31]:
merged_melb_subs = merged_melb_subs.loc[merged_melb_subs['year'] == 2020,:]

In [32]:
merged_melb_subs

,year,lga,suburb,postcode,total_incidents
11,2020,banyule,bellfield,3081,163
21,2020,banyule,briar hill,3088,76
58,2020,banyule,bundoora,3083,332
59,2020,darebin,bundoora,3083,354
60,2020,darebin,bundoora,3086,78
...,...,...,...,...,...
3721,2020,yarra ranges,sherbrooke,3789,7
3731,2020,yarra ranges,tecoma,3160,97
3741,2020,yarra ranges,the patch,3792,9
3751,2020,yarra ranges,upwey,3158,227


### grouping by lga suburb and postcode for sum of total incidnets 

In [33]:
merged_group = pd.DataFrame(merged_melb_subs.groupby(['lga','suburb','postcode'])['total_incidents'].sum())

In [34]:
merged_group

total_incidents
lga          suburb               postcode                 
banyule      bellfield            3081                  163
             briar hill           3088                   76
             bundoora             3083                  332
             eaglemont            3084                   91
             eltham               3095                    1
...                                                     ...
yarra ranges tecoma               3160                   97
             the patch            3792                    9
             upper ferntree gully 3156                   20
             upwey                3158                  227
             wandin north         3139                   66

[375 rows x 1 columns]

### creating list of numbers to give ids for local government areas

In [35]:
len(list(merged_melb_subs['lga'].unique()))

31

In [36]:
numbers = [x for x in range(1,32)]

In [37]:
lga_df = pd.DataFrame(merged_melb_subs['lga'].unique())
lga_df['lga'] = numbers

# id and lga table

In [38]:
lga_df.columns = ['lga','lga_id']
lga_df['lga'] = lga_df['lga'].str.lower()

In [39]:
lga_df

,lga,lga_id
0,banyule,1
1,darebin,2
2,whittlesea,3
3,nillumbik,4
4,bayside,5
5,glen eira,6
6,kingston,7
7,boroondara,8
8,whitehorse,9
9,stonnington,10


In [40]:
lga_df.to_csv('01_Outputs/lga_id_table.csv', index = False)

In [ ]:
lga_df.to_csv('../004_Load_SQL/01_Inputs/lga_id_table.csv', index = False)

### resetting index

In [41]:
merged_group_reset = merged_group.reset_index()

### adding lga id to main table

In [42]:
merged_group_reset=merged_group_reset.merge(lga_df, on = 'lga')

dropping NaN values 

In [52]:
merged_group_reset.dropna(how = 'any')

,lga,suburb,postcode,total_incidents,lga_id
0,banyule,bellfield,3081,163,1
1,banyule,briar hill,3088,76,1
2,banyule,bundoora,3083,332,1
3,banyule,eaglemont,3084,91,1
4,banyule,eltham,3095,1,1
...,...,...,...,...,...
370,yarra ranges,tecoma,3160,97,19
371,yarra ranges,the patch,3792,9,19
372,yarra ranges,upper ferntree gully,3156,20,19
373,yarra ranges,upwey,3158,227,19


# suburb postcode table

In [44]:
sub_postcode = merged_group_reset[['postcode','suburb']]
sub_postcode

,postcode,suburb
0,3081,bellfield
1,3088,briar hill
2,3083,bundoora
3,3084,eaglemont
4,3095,eltham
...,...,...
370,3160,tecoma
371,3792,the patch
372,3156,upper ferntree gully
373,3158,upwey


In [45]:
type(sub_postcode['postcode'][0])

numpy.int64

In [53]:
sub_postcode['postcode'].value_counts()

3095    5
3195    5
3037    5
3977    5
3023    5
       ..
3787    1
3789    1
3791    1
3792    1
3072    1
Name: postcode, Length: 196, dtype: int64

In [46]:
sub_postcode.to_csv('01_Outputs/sub_postcode_table.csv', index = False)

In [ ]:
lga_df.to_csv('../004_Load_SQL/01_Inputs/sub_postcode_table.csv', index = False)


# linking table

In [51]:
linking_table = merged_group_reset[['lga_id','postcode']]
linking_table

,lga_id,postcode
0,1,3081
1,1,3088
2,1,3083
3,1,3084
4,1,3095
...,...,...
370,19,3160
371,19,3792
372,19,3156
373,19,3158


In [48]:
linking_table.to_csv('01_Outputs/linking_table.csv', index = False)

In [ ]:
lga_df.to_csv('../004_Load_SQL/01_Inputs/linking_table.csv', index = False)